In [ ]:
pip install -U transformers datasets seqeval scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f4093fbe2537c3975d3ee74dbbe2e403937a3ae960b8bbf80b999f4ff85651a7
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency res

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive


Mounted at /content/gdrive
/content/gdrive/MyDrive


# Predict using pretrained KindLab Bert

In [ ]:
import json
import random
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline
import torch
from sklearn.metrics import accuracy_score


In [ ]:
# Load KindLab model and tokenizer
model_name = "KindLab/bert-deid"
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # since model has no tokenizer
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
# Load data
import json
from collections import defaultdict
datapath = '/content/gdrive/MyDrive/full_discharge_summaries_and_annotations/'
with open(datapath+"discharge_summaries.json", "r") as f:
    summaries = json.load(f)
text_by_docid = {item["document_id"]: item["text"] for item in summaries}
with open(datapath+"annotations.json", "r") as f:
    annotations = json.load(f)

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
# Step 3: Group annotations by document_id
annotations_by_doc = defaultdict(list)
for ann in annotations:
    doc_id = ann["document_id"]
    annotations_by_doc[doc_id].append({
        "start": ann["start"],
        "end": ann["stop"],
        "label": (ann["entity_type"].replace("IDNUM", "ID")).replace("PHONE", "CONTACT")
    })

# Step 4: Prepare examples
examples = []
for doc_id, labels in annotations_by_doc.items():
    if doc_id not in text_by_docid:
        continue
    text = text_by_docid[doc_id]
    examples.append((text, labels))

# Step 5: Split train/test
train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)

In [ ]:
# Step 7: Create flat label list
label_list = ["O", "AGE", "CONTACT", "DATE", "ID", "LOCATION", "NAME", "PROFESSION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
label_list
label2id

{'O': 0,
 'AGE': 1,
 'CONTACT': 2,
 'DATE': 3,
 'ID': 4,
 'LOCATION': 5,
 'NAME': 6,
 'PROFESSION': 7}

In [ ]:
# Step 8: Align labels with tokens
def align_labels_with_tokens(text, labels, tokenizer):
    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    labels_aligned = ["O"] * len(tokenized_inputs["input_ids"])
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for entity in labels:
        start, end, label = entity["start"], entity["end"], entity["label"]
        for i, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start >= end:
                break
            if offset_end > start and offset_start < end:
                labels_aligned[i] = label.replace(" ", "_")

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": [label2id.get(lbl, 0) for lbl in labels_aligned]
    }

In [ ]:
# Step 9: Tokenize and align the full dataset
train_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in train_data])
test_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in test_data])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

small_test_dataset = dataset["test"].select(range(100))
# Set up DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")
test_dataloader = DataLoader(small_test_dataset, batch_size=8, collate_fn=data_collator)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Stack all batches
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Decode predictions
true_predictions = []
true_labels = []

for prediction, label in zip(all_preds, all_labels):
    temp_pred = []
    temp_label = []
    for p, l in zip(prediction, label):
        if l == -100:
            continue
        temp_pred.append(id2label[p])
        temp_label.append(id2label[l])
    true_predictions.append(temp_pred)
    true_labels.append(temp_label)

Evaluating: 100%|██████████| 13/13 [00:03<00:00,  4.16it/s]


In [ ]:
# Compute Metrics
print("Metrics computed using seqeval:")
print("Accuracy:", accuracy_score(true_labels, true_predictions))
print("Precision:", precision_score(true_labels, true_predictions))
print("Recall:", recall_score(true_labels, true_predictions))
print("F1:", f1_score(true_labels, true_predictions))
print("\nDetailed Classification Report:\n")
print(classification_report(true_labels, true_predictions))

Metrics computed using seqeval:
Accuracy: 0.96646484375
Precision: 0.5747422680412371
Recall: 0.7619589977220956


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ID seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AGE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

F1: 0.6552399608227227

Detailed Classification Report:



/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         AME       0.63      0.95      0.76       200
         ATE       1.00      1.00      1.00       300
           D       1.00      1.00      1.00       100
          GE       0.39      1.00      0.57        78
     OCATION       0.00      0.00      0.00       100
      ONTACT       0.00      0.00      0.00       100
   ROFESSION       0.00      0.00      0.00         0

   micro avg       0.57      0.76      0.66       878
   macro avg       0.43      0.57      0.48       878
weighted avg       0.63      0.76      0.68       878



In [ ]:
# Prepare NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Device set to use cuda:0


In [ ]:
# Compare predictions with annotations
for doc in summaries[:5]:  # Compare first 5 examples
    doc_id = doc["document_id"]
    text = doc["text"]
    print(f"\n================== {doc_id} ==================")
    print(f"Text Sample:\n{text[:400]}...\n")

    # Ground truth annotations
    print("Ground Truth Annotations:")
    for ann in annotations_by_doc.get(doc_id, []):
        snippet = text[ann['start']:ann['end']]
        print(f"{ann['label']}: '{snippet}' (start: {ann['start']}, end: {ann['end']})")

    # Model predictions
    print("\nModel Predictions:")
    preds = ner(text)
    for pred in preds:
        print(f"{pred['entity_group']}: '{pred['word']}' (start: {pred['start']}, end: {pred['end']})")

    print("="*60)


================== ex_ds_1 ==================
Text Sample:
Name: Ashley Wolfe    Unit No: 1110277
Admission Date: 23/08/2023    Discharge Date: 25/08/2023
Date of Birth: 13/07/2009    Age: 57    Sex: F
Service: Paediatrics
Attending: Paula Sutton

Chief Complaint: Patient presented with complaints relevant to paediatrics evaluation.

History of Present Illness:
Ashley Wolfe, a 57-year-old logistics and distribution manager from Robertburgh, KS, was admitt...

Ground Truth Annotations:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
AGE: '57' (start: 130, end: 132)
DATE: '23/08/2023' (start: 55, end: 65)
DATE: '25/08/2023' (start: 85, end: 95)
DATE: '13/07/2009' (start: 111, end: 121)
ID: '1110277' (start: 31, end: 38)
LOCATION: 'Robertburgh, KS' (start: 373, end: 388)
CONTACT: '053-606-5681x5409' (start: 1665, end: 1682)
NAME: 'Paula Sutton' (start: 175, end: 187)

Model Predictions:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
ID: '1110277' (start: 31, end: 38)
DATE: '23 / 08 / 2023' (start: 55

# Finetune KindLab with Synthetic Data

In [ ]:
import json
import random
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [ ]:
# Step 1: Load discharge summaries
import json
from collections import defaultdict
datapath = '/content/gdrive/MyDrive/full_discharge_summaries_and_annotations/'
with open(datapath+"discharge_summaries.json", "r") as f:
    summaries = json.load(f)
text_by_docid = {item["document_id"]: item["text"] for item in summaries}

# Step 2: Load annotations
with open(datapath+"annotations.json", "r") as f:
    annotations = json.load(f)

In [ ]:
# Step 3: Group annotations by document_id
annotations_by_doc = defaultdict(list)
for ann in annotations:
    doc_id = ann["document_id"]
    annotations_by_doc[doc_id].append({
        "start": ann["start"],
        "end": ann["stop"],
        "label": (ann["entity_type"].replace("IDNUM", "ID")).replace("PHONE", "CONTACT")
    })

# Step 4: Prepare examples
examples = []
for doc_id, labels in annotations_by_doc.items():
    if doc_id not in text_by_docid:
        continue
    text = text_by_docid[doc_id]
    examples.append((text, labels))

# Step 5: Split train/test
train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)

In [ ]:
# Step 6: Load tokenizer
model_checkpoint = "KindLab/bert-deid"
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Step 7: Create flat label list
label_list = ["O", "AGE", "CONTACT", "DATE", "ID", "LOCATION", "NAME", "PROFESSION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# Step 8: Align labels with tokens
def align_labels_with_tokens(text, labels, tokenizer):
    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    labels_aligned = ["O"] * len(tokenized_inputs["input_ids"])
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for entity in labels:
        start, end, label = entity["start"], entity["end"], entity["label"]
        for i, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start >= end:
                break
            if offset_end > start and offset_start < end:
                labels_aligned[i] = label.replace(" ", "_")

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": [label2id.get(lbl, 0) for lbl in labels_aligned]
    }

In [ ]:
# Step 9: Tokenize dataset
train_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in train_data])
test_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in test_data])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Step 10: Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)


config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
!pip install evaluate
from transformers import TrainerCallback
from evaluate import load

# Load metrics
seqeval = load("seqeval")

# Custom compute_metrics function
def compute_metrics(predictions):
    preds, labels = predictions
    preds = preds.argmax(-1)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
# Step 11: Setup Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

# Step 12: Define training arguments
training_args = TrainingArguments(
    output_dir="./kindlab_bert_deid_finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    remove_unused_columns=False
)

# Step 13: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 14: Train
trainer.train()

# Step 15: Save model
trainer.save_model("./kindlab_bert_deid_finetuned")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samyakjainuiuc (samyakjainuiuc-university-of-illionis-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
10,0.108600
20,0.005900
30,0.002500
40,0.001200
50,0.000900
60,0.000700
70,0.000900
80,0.001100
90,0.001000
100,0.000900


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

small_test_dataset = dataset["test"].select(range(100))
# Set up DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")
test_dataloader = DataLoader(small_test_dataset, batch_size=8, collate_fn=data_collator)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Stack all batches
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Decode predictions
true_predictions = []
true_labels = []

for prediction, label in zip(all_preds, all_labels):
    temp_pred = []
    temp_label = []
    for p, l in zip(prediction, label):
        if l == -100:
            continue
        temp_pred.append(id2label[p])
        temp_label.append(id2label[l])
    true_predictions.append(temp_pred)
    true_labels.append(temp_label)

Evaluating: 100%|██████████| 13/13 [00:02<00:00,  4.43it/s]


In [ ]:
print("Metrics computed using seqeval:")
print("Accuracy:", accuracy_score(true_labels, true_predictions))
print("Precision:", precision_score(true_labels, true_predictions))
print("Recall:", recall_score(true_labels, true_predictions))
print("F1:", f1_score(true_labels, true_predictions))
print("\nDetailed Classification Report:\n")
print(classification_report(true_labels, true_predictions))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ID seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AGE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

Metrics computed using seqeval:
Accuracy: 0.9996875
Precision: 0.9853603603603603
Recall: 0.9965831435079726
F1: 0.9909399773499433

Detailed Classification Report:

              precision    recall  f1-score   support

         AME       1.00      1.00      1.00       200
         ATE       1.00      1.00      1.00       300
           D       1.00      1.00      1.00       100
          GE       0.85      0.96      0.90        78
     OCATION       1.00      1.00      1.00       100
      ONTACT       1.00      1.00      1.00       100

   micro avg       0.99      1.00      0.99       878
   macro avg       0.98      0.99      0.98       878
weighted avg       0.99      1.00      0.99       878



In [ ]:
# Prepare NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
# Compare predictions with annotations
for doc in summaries[:5]:  # Compare first 5 examples
    doc_id = doc["document_id"]
    text = doc["text"]
    print(f"\n================== {doc_id} ==================")
    print(f"Text Sample:\n{text[:400]}...\n")

    # Ground truth annotations
    print("Ground Truth Annotations:")
    for ann in annotations_by_doc.get(doc_id, []):
        snippet = text[ann['start']:ann['end']]
        print(f"{ann['label']}: '{snippet}' (start: {ann['start']}, end: {ann['end']})")

    # Model predictions
    print("\nModel Predictions:")
    preds = ner(text)
    for pred in preds:
        print(f"{pred['entity_group']}: '{pred['word']}' (start: {pred['start']}, end: {pred['end']})")

    print("="*60)

# Finetune KindLab with Synthetic Data with best set of hyperparameters

In [ ]:
import json
import random
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [ ]:
# Step 1: Load discharge summaries
import json
from collections import defaultdict
datapath = '/content/gdrive/MyDrive/full_discharge_summaries_and_annotations/'
with open(datapath+"discharge_summaries.json", "r") as f:
    summaries = json.load(f)
text_by_docid = {item["document_id"]: item["text"] for item in summaries}

# Step 2: Load annotations
with open(datapath+"annotations.json", "r") as f:
    annotations = json.load(f)

In [ ]:
# Step 3: Group annotations by document_id
annotations_by_doc = defaultdict(list)
for ann in annotations:
    doc_id = ann["document_id"]
    annotations_by_doc[doc_id].append({
        "start": ann["start"],
        "end": ann["stop"],
        "label": (ann["entity_type"].replace("IDNUM", "ID")).replace("PHONE", "CONTACT")
    })

# Step 4: Prepare examples
examples = []
for doc_id, labels in annotations_by_doc.items():
    if doc_id not in text_by_docid:
        continue
    text = text_by_docid[doc_id]
    examples.append((text, labels))

# Step 5: Split train/test
train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)

In [ ]:
# Step 6: Load tokenizer
model_checkpoint = "KindLab/bert-deid"
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Step 7: Create flat label list
label_list = ["O", "AGE", "CONTACT", "DATE", "ID", "LOCATION", "NAME", "PROFESSION"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# Step 8: Align labels with tokens
def align_labels_with_tokens(text, labels, tokenizer):
    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    labels_aligned = ["O"] * len(tokenized_inputs["input_ids"])
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for entity in labels:
        start, end, label = entity["start"], entity["end"], entity["label"]
        for i, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start >= end:
                break
            if offset_end > start and offset_start < end:
                labels_aligned[i] = label.replace(" ", "_")

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": [label2id.get(lbl, 0) for lbl in labels_aligned]
    }

In [ ]:
# Step 9: Tokenize dataset
train_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in train_data])
test_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in test_data])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

# Step 10: Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)


config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [ ]:
!pip install evaluate
from transformers import TrainerCallback
from evaluate import load

# Load metrics
seqeval = load("seqeval")

# Custom compute_metrics function
def compute_metrics(predictions):
    preds, labels = predictions
    preds = preds.argmax(-1)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
# Step 11: Setup Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

# Step 12: Define training arguments
# training_args = TrainingArguments(
#     output_dir="./kindlab_bert_deid_finetuned",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=5,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_steps=200,
#     save_total_limit=2,
#     remove_unused_columns=False
# )
# Step 12: Define training arguments (best hyperparameters based on tuning)
training_args = TrainingArguments(
    output_dir="./kindlab_bert_deid_finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    remove_unused_columns=False
)

# Step 13: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 14: Train
trainer.train()

# Step 15: Save model
trainer.save_model("./kindlab_bert_deid_finetuned")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samyakjainuiuc (samyakjainuiuc-university-of-illionis-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
10,0.136600
20,0.019300
30,0.007300
40,0.004500
50,0.002200
60,0.001800
70,0.001200
80,0.001300
90,0.001200
100,0.001100


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

small_test_dataset = dataset["test"].select(range(100))
# Set up DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")
test_dataloader = DataLoader(small_test_dataset, batch_size=8, collate_fn=data_collator)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Stack all batches
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Decode predictions
true_predictions = []
true_labels = []

for prediction, label in zip(all_preds, all_labels):
    temp_pred = []
    temp_label = []
    for p, l in zip(prediction, label):
        if l == -100:
            continue
        temp_pred.append(id2label[p])
        temp_label.append(id2label[l])
    true_predictions.append(temp_pred)
    true_labels.append(temp_label)

Evaluating: 100%|██████████| 13/13 [00:03<00:00,  4.23it/s]


In [ ]:
print("Metrics computed using seqeval:")
print("Accuracy:", accuracy_score(true_labels, true_predictions))
print("Precision:", precision_score(true_labels, true_predictions))
print("Recall:", recall_score(true_labels, true_predictions))
print("F1:", f1_score(true_labels, true_predictions))
print("\nDetailed Classification Report:\n")
print(classification_report(true_labels, true_predictions))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ID seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: AGE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:17

Metrics computed using seqeval:
Accuracy: 0.99970703125
Precision: 0.9853768278965129
Recall: 0.9977220956719818
F1: 0.9915110356536503

Detailed Classification Report:

              precision    recall  f1-score   support

         AME       1.00      1.00      1.00       200
         ATE       1.00      1.00      1.00       300
           D       1.00      1.00      1.00       100
          GE       0.85      0.97      0.91        78
     OCATION       1.00      1.00      1.00       100
      ONTACT       1.00      1.00      1.00       100

   micro avg       0.99      1.00      0.99       878
   macro avg       0.98      1.00      0.99       878
weighted avg       0.99      1.00      0.99       878



In [ ]:
# Prepare NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cuda:0


In [ ]:
# Compare predictions with annotations
for doc in summaries[:5]:  # Compare first 5 examples
    doc_id = doc["document_id"]
    text = doc["text"]
    print(f"\n================== {doc_id} ==================")
    print(f"Text Sample:\n{text[:400]}...\n")

    # Ground truth annotations
    print("Ground Truth Annotations:")
    for ann in annotations_by_doc.get(doc_id, []):
        snippet = text[ann['start']:ann['end']]
        print(f"{ann['label']}: '{snippet}' (start: {ann['start']}, end: {ann['end']})")

    # Model predictions
    print("\nModel Predictions:")
    preds = ner(text)
    for pred in preds:
        print(f"{pred['entity_group']}: '{pred['word']}' (start: {pred['start']}, end: {pred['end']})")

    print("="*60)


================== ex_ds_1 ==================
Text Sample:
Name: Ashley Wolfe    Unit No: 1110277
Admission Date: 23/08/2023    Discharge Date: 25/08/2023
Date of Birth: 13/07/2009    Age: 57    Sex: F
Service: Paediatrics
Attending: Paula Sutton

Chief Complaint: Patient presented with complaints relevant to paediatrics evaluation.

History of Present Illness:
Ashley Wolfe, a 57-year-old logistics and distribution manager from Robertburgh, KS, was admitt...

Ground Truth Annotations:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
AGE: '57' (start: 130, end: 132)
DATE: '23/08/2023' (start: 55, end: 65)
DATE: '25/08/2023' (start: 85, end: 95)
DATE: '13/07/2009' (start: 111, end: 121)
ID: '1110277' (start: 31, end: 38)
LOCATION: 'Robertburgh, KS' (start: 373, end: 388)
CONTACT: '053-606-5681x5409' (start: 1665, end: 1682)
NAME: 'Paula Sutton' (start: 175, end: 187)

Model Predictions:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
ID: '1110277' (start: 31, end: 38)
DATE: '23 / 08 / 2023' (start: 55

# Training Fresh on Synthetic Data

In [ ]:
import json
import random
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from collections import defaultdict

In [ ]:
# Load data
import json
from collections import defaultdict
datapath = '/content/gdrive/MyDrive/full_discharge_summaries_and_annotations/'
with open(datapath+"discharge_summaries.json", "r") as f:
    summaries = json.load(f)
text_by_docid = {item["document_id"]: item["text"] for item in summaries}

with open(datapath+"annotations.json", "r") as f:
    annotations = json.load(f)

# Step 3: Group annotations by document_id
annotations_by_doc = defaultdict(list)
for ann in annotations:
    doc_id = ann["document_id"]
    annotations_by_doc[doc_id].append({
        "start": ann["start"],
        "end": ann["stop"],  # correcting stop -> end
        "label": (ann["entity_type"].replace("IDNUM", "ID")).replace("PHONE", "CONTACT")  # Normalize label
    })

In [ ]:
# Step 4: Prepare examples (text + labels)
examples = []
for doc_id, labels in annotations_by_doc.items():
    if doc_id not in text_by_docid:
        continue
    text = text_by_docid[doc_id]
    examples.append((text, labels))

# Step 5: Train/test split
train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)


In [ ]:
# Step 6: Load tokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Step 7: Create label list
label_list = list({label["label"].replace(" ", "_") for _, anns in examples for label in anns})
label_list = ["O"] + ["B-" + label for label in label_list] + ["I-" + label for label in label_list]

In [ ]:
label_list

['O',
 'B-DATE',
 'B-LOCATION',
 'B-ID',
 'B-NAME',
 'B-AGE',
 'B-CONTACT',
 'I-DATE',
 'I-LOCATION',
 'I-ID',
 'I-NAME',
 'I-AGE',
 'I-CONTACT']

In [ ]:
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
# Step 8: Align labels with tokens
def align_labels_with_tokens(text, labels, tokenizer):
    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    labels_aligned = ["O"] * len(tokenized_inputs["input_ids"])
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for entity in labels:
        start, end, label = entity["start"], entity["end"], entity["label"]
        for i, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start >= end:
                break
            if offset_end > start and offset_start < end:
                if labels_aligned[i] == "O":
                    labels_aligned[i] = "B-" + label.replace(" ", "_")
                else:
                    labels_aligned[i] = "I-" + label.replace(" ", "_")

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": [label2id.get(lbl, 0) for lbl in labels_aligned]
    }

In [ ]:
# Step 9: Tokenize and align the full dataset
train_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in train_data])
test_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in test_data])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# Step 10: Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Step 11: Set up Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 12: Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_base_cased_finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    remove_unused_columns=False
)

# Step 13: Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

# Step 14: Train the model
trainer.train()

# Step 15: Save the model
trainer.save_model("./bert_base_cased_finetuned")


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
20,0.362000
40,0.007400
60,0.004700
80,0.004400
100,0.003500
120,0.002900
140,0.004000
160,0.003800
180,0.003000
200,0.003700


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
#Evaluate Metrics
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

small_test_dataset = dataset["test"].select(range(100))
# Set up DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")
test_dataloader = DataLoader(small_test_dataset, batch_size=8, collate_fn=data_collator)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Stack all batches
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Decode predictions
true_predictions = []
true_labels = []

for prediction, label in zip(all_preds, all_labels):
    temp_pred = []
    temp_label = []
    for p, l in zip(prediction, label):
        if l == -100:
            continue
        temp_pred.append(id2label[p])
        temp_label.append(id2label[l])
    true_predictions.append(temp_pred)
    true_labels.append(temp_label)

Evaluating: 100%|██████████| 13/13 [00:02<00:00,  4.40it/s]


In [ ]:
# Compute Metrics
print("Metrics computed using seqeval:")
print("Accuracy:", accuracy_score(true_labels, true_predictions))
print("Precision:", precision_score(true_labels, true_predictions))
print("Recall:", recall_score(true_labels, true_predictions))
print("F1:", f1_score(true_labels, true_predictions))
print("\nDetailed Classification Report:\n")
print(classification_report(true_labels, true_predictions))

Metrics computed using seqeval:
Accuracy: 0.99921875
Precision: 0.989979025868096
Recall: 0.9960140679953107
F1: 0.9929873772791025

Detailed Classification Report:

              precision    recall  f1-score   support

         AGE       0.85      0.96      0.90        78
     CONTACT       1.00      1.00      1.00      1128
        DATE       0.99      0.99      0.99      1696
          ID       0.98      0.99      0.98       419
    LOCATION       1.00      1.00      1.00       476
        NAME       1.00      1.00      1.00       468

   micro avg       0.99      1.00      0.99      4265
   macro avg       0.97      0.99      0.98      4265
weighted avg       0.99      1.00      0.99      4265



In [ ]:
# Prepare NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Compare predictions with annotations
for doc in summaries[:5]:  # Compare first 5 examples
    doc_id = doc["document_id"]
    text = doc["text"]
    print(f"\n================== {doc_id} ==================")
    print(f"Text Sample:\n{text[:400]}...\n")

    # Ground truth annotationsS
    print("Ground Truth Annotations:")
    for ann in annotations_by_doc.get(doc_id, []):
        snippet = text[ann['start']:ann['end']]
        print(f"{ann['label']}: '{snippet}' (start: {ann['start']}, end: {ann['end']})")

    # Model predictions
    print("\nModel Predictions:")
    preds = ner(text)
    for pred in preds:
        print(f"{pred['entity_group']}: '{pred['word']}' (start: {pred['start']}, end: {pred['end']})")

    print("="*60)

Device set to use cuda:0



================== ex_ds_1 ==================
Text Sample:
Name: Ashley Wolfe    Unit No: 1110277
Admission Date: 23/08/2023    Discharge Date: 25/08/2023
Date of Birth: 13/07/2009    Age: 57    Sex: F
Service: Paediatrics
Attending: Paula Sutton

Chief Complaint: Patient presented with complaints relevant to paediatrics evaluation.

History of Present Illness:
Ashley Wolfe, a 57-year-old logistics and distribution manager from Robertburgh, KS, was admitt...

Ground Truth Annotations:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
AGE: '57' (start: 130, end: 132)
DATE: '23/08/2023' (start: 55, end: 65)
DATE: '25/08/2023' (start: 85, end: 95)
DATE: '13/07/2009' (start: 111, end: 121)
ID: '1110277' (start: 31, end: 38)
LOCATION: 'Robertburgh, KS' (start: 373, end: 388)
CONTACT: '053-606-5681x5409' (start: 1665, end: 1682)
NAME: 'Paula Sutton' (start: 175, end: 187)

Model Predictions:
NAME: 'Ashley' (start: 6, end: 12)
NAME: 'Wolfe' (start: 13, end: 18)
ID: '111' (start: 31, end: 34)
ID: '##0

# Training Fresh on Synthetic Data with best set of hyperparameters

In [ ]:
import json
import random
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from collections import defaultdict

In [ ]:
# Load data
import json
from collections import defaultdict
datapath = '/content/gdrive/MyDrive/full_discharge_summaries_and_annotations/'
with open(datapath+"discharge_summaries.json", "r") as f:
    summaries = json.load(f)
text_by_docid = {item["document_id"]: item["text"] for item in summaries}

with open(datapath+"annotations.json", "r") as f:
    annotations = json.load(f)

# Step 3: Group annotations by document_id
annotations_by_doc = defaultdict(list)
for ann in annotations:
    doc_id = ann["document_id"]
    annotations_by_doc[doc_id].append({
        "start": ann["start"],
        "end": ann["stop"],  # correcting stop -> end
        "label": (ann["entity_type"].replace("IDNUM", "ID")).replace("PHONE", "CONTACT")  # Normalize label
    })

In [ ]:
# Step 4: Prepare examples (text + labels)
examples = []
for doc_id, labels in annotations_by_doc.items():
    if doc_id not in text_by_docid:
        continue
    text = text_by_docid[doc_id]
    examples.append((text, labels))

# Step 5: Train/test split
train_data, test_data = train_test_split(examples, test_size=0.2, random_state=42)


In [ ]:
# Step 6: Load tokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Step 7: Create label list
label_list = list({label["label"].replace(" ", "_") for _, anns in examples for label in anns})
label_list = ["O"] + ["B-" + label for label in label_list] + ["I-" + label for label in label_list]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
label_list

['O',
 'B-ID',
 'B-DATE',
 'B-NAME',
 'B-CONTACT',
 'B-LOCATION',
 'B-AGE',
 'I-ID',
 'I-DATE',
 'I-NAME',
 'I-CONTACT',
 'I-LOCATION',
 'I-AGE']

In [ ]:
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
# Step 8: Align labels with tokens
def align_labels_with_tokens(text, labels, tokenizer):
    tokenized_inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    labels_aligned = ["O"] * len(tokenized_inputs["input_ids"])
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for entity in labels:
        start, end, label = entity["start"], entity["end"], entity["label"]
        for i, (offset_start, offset_end) in enumerate(offset_mapping):
            if offset_start >= end:
                break
            if offset_end > start and offset_start < end:
                if labels_aligned[i] == "O":
                    labels_aligned[i] = "B-" + label.replace(" ", "_")
                else:
                    labels_aligned[i] = "I-" + label.replace(" ", "_")

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": [label2id.get(lbl, 0) for lbl in labels_aligned]
    }

In [ ]:
# Step 9: Tokenize and align the full dataset
train_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in train_data])
test_dataset = Dataset.from_list([align_labels_with_tokens(x[0], x[1], tokenizer) for x in test_data])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# Step 10: Load model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Step 11: Set up Data Collator
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 12: Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_base_cased_finetuned",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    remove_unused_columns=False
)

# Step 13: Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator
)

# Step 14: Train the model
trainer.train()

# Step 15: Save the model
trainer.save_model("./bert_base_cased_finetuned")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: samyakjainuiuc (samyakjainuiuc-university-of-illionis-urbana-champaign) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
20,0.690600
40,0.040400
60,0.008700
80,0.006400
100,0.005200
120,0.005000


In [ ]:
#Evaluate Metrics
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorForTokenClassification
from seqeval.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

small_test_dataset = dataset["test"].select(range(100))
# Set up DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True, return_tensors="pt")
test_dataloader = DataLoader(small_test_dataset, batch_size=8, collate_fn=data_collator)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = logits.argmax(dim=-1)

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Stack all batches
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

# Decode predictions
true_predictions = []
true_labels = []

for prediction, label in zip(all_preds, all_labels):
    temp_pred = []
    temp_label = []
    for p, l in zip(prediction, label):
        if l == -100:
            continue
        temp_pred.append(id2label[p])
        temp_label.append(id2label[l])
    true_predictions.append(temp_pred)
    true_labels.append(temp_label)

Evaluating: 100%|██████████| 13/13 [00:03<00:00,  4.21it/s]


In [ ]:
# Compute Metrics
print("Metrics computed using seqeval:")
print("Accuracy:", accuracy_score(true_labels, true_predictions))
print("Precision:", precision_score(true_labels, true_predictions))
print("Recall:", recall_score(true_labels, true_predictions))
print("F1:", f1_score(true_labels, true_predictions))
print("\nDetailed Classification Report:\n")
print(classification_report(true_labels, true_predictions))

Metrics computed using seqeval:
Accuracy: 0.99919921875
Precision: 0.9892923649906891
Recall: 0.9964830011723329
F1: 0.9928746641747459

Detailed Classification Report:

              precision    recall  f1-score   support

         AGE       0.83      0.99      0.90        78
     CONTACT       1.00      1.00      1.00      1128
        DATE       0.99      0.99      0.99      1696
          ID       0.98      0.99      0.98       419
    LOCATION       1.00      1.00      1.00       476
        NAME       1.00      1.00      1.00       468

   micro avg       0.99      1.00      0.99      4265
   macro avg       0.97      1.00      0.98      4265
weighted avg       0.99      1.00      0.99      4265



In [ ]:
from transformers import pipeline
# Prepare NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Compare predictions with annotations
for doc in summaries[:5]:  # Compare first 5 examples
    doc_id = doc["document_id"]
    text = doc["text"]
    print(f"\n================== {doc_id} ==================")
    print(f"Text Sample:\n{text[:400]}...\n")

    # Ground truth annotationsS
    print("Ground Truth Annotations:")
    for ann in annotations_by_doc.get(doc_id, []):
        snippet = text[ann['start']:ann['end']]
        print(f"{ann['label']}: '{snippet}' (start: {ann['start']}, end: {ann['end']})")

    # Model predictions
    print("\nModel Predictions:")
    preds = ner(text)
    for pred in preds:
        print(f"{pred['entity_group']}: '{pred['word']}' (start: {pred['start']}, end: {pred['end']})")

    print("="*60)

Device set to use cuda:0



================== ex_ds_1 ==================
Text Sample:
Name: Ashley Wolfe    Unit No: 1110277
Admission Date: 23/08/2023    Discharge Date: 25/08/2023
Date of Birth: 13/07/2009    Age: 57    Sex: F
Service: Paediatrics
Attending: Paula Sutton

Chief Complaint: Patient presented with complaints relevant to paediatrics evaluation.

History of Present Illness:
Ashley Wolfe, a 57-year-old logistics and distribution manager from Robertburgh, KS, was admitt...

Ground Truth Annotations:
NAME: 'Ashley Wolfe' (start: 6, end: 18)
AGE: '57' (start: 130, end: 132)
DATE: '23/08/2023' (start: 55, end: 65)
DATE: '25/08/2023' (start: 85, end: 95)
DATE: '13/07/2009' (start: 111, end: 121)
ID: '1110277' (start: 31, end: 38)
LOCATION: 'Robertburgh, KS' (start: 373, end: 388)
CONTACT: '053-606-5681x5409' (start: 1665, end: 1682)
NAME: 'Paula Sutton' (start: 175, end: 187)

Model Predictions:
NAME: 'Ashley' (start: 6, end: 12)
NAME: 'Wolfe' (start: 13, end: 18)
ID: '111' (start: 31, end: 34)
ID: '##0